In [1]:
%load_ext autoreload
%autoreload 2

In [27]:
# utility functions for loading data 
from models import *
from training_utils import *
from data_utils import *
import argparse


os.environ['fairness_data'] = '/Users/neelguha/Dropbox/NeelResearch/fairness/data/'
if 'fairness_data' in os.environ:
    DATA_DIR = os.environ['fairness_data']
else:
    raise Exception("Please set environment variable: fairness_data to point towards the data directory")

In [17]:
def evaluate(dataset, train_errors, train_violations, test_errors, test_violations, test_df, label_column):
    print("=== TRAINING ERROR ====")
    print("Train Error", train_errors[-1])
    print("Maximum Train Violation", max(train_violations[-1]))
    print()
    print("=== TEST ERROR ====")
    print("Maximum Test Error", test_errors[-1])
    print("Maximum Test Violation", max(test_violations[-1]))
    print()
    print("=== OVERALL PERFORMANCE ====")
    print("Baseline overall accuracy: %f" %
          (1 - error_rate(test_df['predictions'], test_df['label'])))
    print("Baseline overall TPR: %f" % tpr(test_df, label_column))
    print()
    print("=== ACCURACY SUBSETS ====")
    subsets = get_subsets(dataset)
    for subset in subsets:
        acc = get_acc_subset(test_df, subset)
        print('|'.join(subset), "Accuracy:", acc)

    print()
    print("=== TPR SUBSETS ====")
    for subset in subsets:
        tpr_val = get_tpr_subset(test_df, subset, label_column)
        print('|'.join(subset), "TPR:", tpr_val)

In [32]:
train_df, test_df, feature_names, protected_columns, label_column = get_ipums_income()

In [33]:
# Run baseline
print("Running baseline")
model = LinearModel(feature_names, protected_columns,
                    label_column, None, tpr_max_diff=0.05)
model.build_train_op(0.01, unconstrained=True)

# training_helper returns the list of errors and violations over each epoch.
train_errors, train_violations, test_errors, test_violations = training_helper(
    model, train_df, test_df, 100, label_column, protected_columns, num_iterations_per_loop=326, num_loops=40
)
evaluate(args.dataset, train_errors, train_violations,
         test_errors, test_violations, test_df, label_column)

Running baseline


ZeroDivisionError: float division by zero

In [43]:
_get_error_rate_and_constraints(train_df, model.tpr_max_diff, label_column, protected_columns)

NameError: name '_get_error_rate_and_constraints' is not defined